In [ ]:
#@markdown # Prepare files

!for file in bayes_evalue.py fitted_params.py report_null_model_evalues.py; do \
wget -q https://raw.githubusercontent.com/rcedgar/null_model2/refs/heads/main/py/${file} -O ${file}; \
done

import pandas as pd
from google.colab import files
import io
import ipywidgets as widgets
from IPython.display import display

import math
from bayes_evalue import *

print("Please upload your TSV file.")
uploaded = files.upload()

if not uploaded:
    print("No file uploaded. Please try again.")
filename = next(iter(uploaded))



In [ ]:
#@markdown Select parameters

score_type = "Foldsek E-value" #@param ['Foldsek E-value', 'DALI Z-score', 'TM-score']
column = "3" #@param {type:"string"}
dbsize = "10000" #@param {type:"string"}
dbsize = int(dbsize)
calibration = "scop40c" #@param ['scop40c', 'scop40']

#@markdown ---


In [ ]:
#@markdown Calculate E-values

# 0-based column nr.
colnr = int(column) -1 # 0-based column nr.
refdb = calibration


fin = open(filename, "r")
fout = open("updated_E_values.tsv", "w")

hdr = fin.readline()[:-1] + "\tE-value_null\n"
fout.write(hdr)

if score_type == "TM-score":
	algo = "tm"
	h = 1
	PF = 1
elif score_type == "DALI Z-score":
	algo = "dali"
	h = 0.04
	PF = 0.5
elif score_type == "Foldsek E-value":
	algo = "foldseek"
	h = 0.04
	PF = 0.5
else:
	assert False, "invalid --score " + str(Args.score)

m, c = get_m_c("foldseek", "scop40")
m_c, c_c = get_m_c("foldseek", "scop40c")

def evalue_str(s):
	if score_type == "E":
		Eprime = 0.05*(s**0.5 + 0.1*s**0.25)
		r = dbsize**0.68
		E = r*Eprime
		if refdb == "scop40":
			E *= 2.5
	else:
		E = estimate_FPEPQ(algo, refdb, s, dbsize, PF, h)
	if E > 10:
		return ">10"
	return "%.2g" % E

for line in fin:
	line = line[:-1]
	flds = line.split('\t')
	s = float(flds[colnr])
	line += "\t" + evalue_str(s)
	fout.write(line+ "\n")
	print(line)


In [ ]:
#@markdown Donwload updated table

files.download('updated_E_values.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>